In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder\
.appName('Pyspark Data Handling')\
.getOrCreate()

25/12/22 12:57:25 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [3]:
data = [
    (1, "John Doe", "Bangalore", "2023-01-15", "152.75", "True"),
    (2, "Jane Smith", "Delhi", "2023-05-20", "89.50", "False"),
    (3, "Robert Brown", "Mumbai", "InvalidDate", "200.00", "True"),
    (4, "Linda White", "Kolkata", "2023-02-29", None, "yes"),  # Feb 29 invalid in 2023
    (5, "Mike Green", "Chennai", "2023-08-10", "NaN", "1"),  # NaN needs handling
    (6, "Sarah Blue", "Hyderabad", "InvalidDate", "300.40", "NO")
]

# Define column names
columns = ["id", "name", "city", "date", "amount", "is_active"]

df = spark.createDataFrame(data,schema = columns)

df.show()

+---+------------+---------+-----------+------+---------+
| id|        name|     city|       date|amount|is_active|
+---+------------+---------+-----------+------+---------+
|  1|    John Doe|Bangalore| 2023-01-15|152.75|     True|
|  2|  Jane Smith|    Delhi| 2023-05-20| 89.50|    False|
|  3|Robert Brown|   Mumbai|InvalidDate|200.00|     True|
|  4| Linda White|  Kolkata| 2023-02-29|  NULL|      yes|
|  5|  Mike Green|  Chennai| 2023-08-10|   NaN|        1|
|  6|  Sarah Blue|Hyderabad|InvalidDate|300.40|       NO|
+---+------------+---------+-----------+------+---------+



In [5]:
df.printSchema()

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- city: string (nullable = true)
 |-- date: string (nullable = true)
 |-- amount: string (nullable = true)
 |-- is_active: string (nullable = true)



In [6]:
#handle Integer Column

In [7]:
df.filter(df.id>3).show()

+---+-----------+---------+-----------+------+---------+
| id|       name|     city|       date|amount|is_active|
+---+-----------+---------+-----------+------+---------+
|  4|Linda White|  Kolkata| 2023-02-29|  NULL|      yes|
|  5| Mike Green|  Chennai| 2023-08-10|   NaN|        1|
|  6| Sarah Blue|Hyderabad|InvalidDate|300.40|       NO|
+---+-----------+---------+-----------+------+---------+



In [8]:
df.withColumn('id_d',df.id*2).show()

+---+------------+---------+-----------+------+---------+----+
| id|        name|     city|       date|amount|is_active|id_d|
+---+------------+---------+-----------+------+---------+----+
|  1|    John Doe|Bangalore| 2023-01-15|152.75|     True|   2|
|  2|  Jane Smith|    Delhi| 2023-05-20| 89.50|    False|   4|
|  3|Robert Brown|   Mumbai|InvalidDate|200.00|     True|   6|
|  4| Linda White|  Kolkata| 2023-02-29|  NULL|      yes|   8|
|  5|  Mike Green|  Chennai| 2023-08-10|   NaN|        1|  10|
|  6|  Sarah Blue|Hyderabad|InvalidDate|300.40|       NO|  12|
+---+------------+---------+-----------+------+---------+----+



In [13]:
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import *

In [14]:
df = df.withColumn('id',col('id').cast(IntegerType()))

In [15]:
df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- city: string (nullable = true)
 |-- date: string (nullable = true)
 |-- amount: string (nullable = true)
 |-- is_active: string (nullable = true)



In [12]:
# String Col

In [17]:
df = df.withColumn('name_upper',upper(df.name))

In [18]:
df.show()

+---+------------+---------+-----------+------+---------+------------+
| id|        name|     city|       date|amount|is_active|  name_upper|
+---+------------+---------+-----------+------+---------+------------+
|  1|    John Doe|Bangalore| 2023-01-15|152.75|     True|    JOHN DOE|
|  2|  Jane Smith|    Delhi| 2023-05-20| 89.50|    False|  JANE SMITH|
|  3|Robert Brown|   Mumbai|InvalidDate|200.00|     True|ROBERT BROWN|
|  4| Linda White|  Kolkata| 2023-02-29|  NULL|      yes| LINDA WHITE|
|  5|  Mike Green|  Chennai| 2023-08-10|   NaN|        1|  MIKE GREEN|
|  6|  Sarah Blue|Hyderabad|InvalidDate|300.40|       NO|  SARAH BLUE|
+---+------------+---------+-----------+------+---------+------------+



In [19]:
df.filter(df.city.startswith('B')).show()

+---+--------+---------+----------+------+---------+----------+
| id|    name|     city|      date|amount|is_active|name_upper|
+---+--------+---------+----------+------+---------+----------+
|  1|John Doe|Bangalore|2023-01-15|152.75|     True|  JOHN DOE|
+---+--------+---------+----------+------+---------+----------+



In [22]:
df =df.withColumn('amount',col('amount').cast('float'))

In [23]:
df_fill = df.fillna({'amount':0})

In [24]:
df_fill.show()

+---+------------+---------+-----------+------+---------+------------+
| id|        name|     city|       date|amount|is_active|  name_upper|
+---+------------+---------+-----------+------+---------+------------+
|  1|    John Doe|Bangalore| 2023-01-15|152.75|     True|    JOHN DOE|
|  2|  Jane Smith|    Delhi| 2023-05-20|  89.5|    False|  JANE SMITH|
|  3|Robert Brown|   Mumbai|InvalidDate| 200.0|     True|ROBERT BROWN|
|  4| Linda White|  Kolkata| 2023-02-29|   0.0|      yes| LINDA WHITE|
|  5|  Mike Green|  Chennai| 2023-08-10|   0.0|        1|  MIKE GREEN|
|  6|  Sarah Blue|Hyderabad|InvalidDate| 300.4|       NO|  SARAH BLUE|
+---+------------+---------+-----------+------+---------+------------+



In [26]:
spark.stop()